In [441]:
'''
DEFINE MASTER FUNCTIONS 
'''

# Census 1 
def define_geography(map_name, gdb, focus_points, focus_point_name, selection_distance, shp_folder): 

    # Clear map 
    mp        = define_map(map_name)
    mp        = clear_map(mp)

    # Export block group selections 
    fc_path   = select_block_groups_by_focus_point(focus_point_name, focus_points, block_groups, selection_distance, mp, overwrite = True, add_to_map = True)
    
    shp_path  = export_shapefiles(fc_path, shp_folder)
    
    return shp_path

# Census 2
def view_acs_codes(json_url, filter_words, blocked_words): 
    
    # Read JSON 
    json_dictionary         = read_census_json(json_url)
    group_dictionary        = return_group_dictionary(json_dictionary)

    # View the contents 
    acs_dataframe          = print_group_dictionary(group_dictionary, filter_words, blocked_words, print_groups = False, print_codes = False)
    
    return json_dictionary, group_dictionary, acs_dataframe 

def return_acs_dictionary(input_group_code, json_dictionary, group_dictionary):
    
    # Find relevant codes 
    variable_codes        = find_variable_codes(input_group_code, group_dictionary)
    code_label_dictionary = create_code_label_dictionary(variable_codes, json_dictionary)
    
    return variable_codes, code_label_dictionary

# Census 3 
def retrieve_acs_json(variable_codes, shp_path, api_key): 
    
    variable_string = create_variable_string(variable_codes)
    api_urls        = construct_api_urls(shp_path, api_key, variable_string)
    json_list       = call_census_api(api_urls)
    
    return json_list

def json_to_dataframe(json_list): 
    
    census_dataframe  = json_list_to_dataframe(json_list)
    census_dataframe  = change_columns_to_integers(census_dataframe)
    census_dataframe  = calculate_percentage_columns(census_dataframe)
    
    return census_dataframe 

def create_geodataframe(shp_path, keep_columns): 
    
    # Create geodataframe 
    geodataframe = gpd.read_file(shp_path)
    
    # Drop unnecessary columns 
    for column in geodataframe.columns: 

        if column not in keep_columns: 
            try: 
                geodataframe = geodataframe.drop(columns = column)
            except: 
                pass
            
    return geodataframe

def merge_dataframe_with_geometry(geodataframe, dataframe, output_shp_path): 
    
    merge_geodataframe = geodataframe.merge(census_dataframe, on = 'GEOID', how = 'left')
    merge_geodataframe.to_file(output_shp_path + ".shp", driver = 'ESRI Shapefile')
    
    print(output_shp_path)
    
    return merge_geodataframe, output_shp_path

In [442]:
##########################################################################################
##########################################################################################
##########################################################################################
##########################################################################################

In [443]:
'''
PARAMETERS
'''

# Imports 
import os 
import pandas as pd
import geopandas as gpd 
import requests 
from pandas import option_context

# Map 
map_name   = 'Census'
add_to_map = True 

# Input Data  
block_groups = r"https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/1"
focus_points = r"C:\DALE\Personal\Minnesota\MyProject\MyProject.gdb\Focus_Points"

# Output Data 
gdb          = r"C:\DALE\Personal\Minnesota\MyProject\MyProject.gdb"
shp_folder   = r"C:\DALE\Personal\Minnesota\MyProject\Shapefiles"
output_shp   = os.path.join(shp_folder, 'Minneapolis_MonthlyHousingCosts')

# Selection criteria 
focus_point_name   = 'SMU Minneapolis'
selection_distance = '10 Miles'

# Variables searching 
filter_words       = []
blocked_words      = []

# API call 
api_key            = r"1749dc1d87964116107a80cd7d76fca300dda59f"
base_url           = r"https://api.census.gov/data/2022/acs/acs5"
json_url           = fr"{base_url}/variables.json"

input_group_code   = 'B08303'

# Geodataframe 
keep_columns       = ['GEOID', 'geometry']

In [444]:
'''
RUN MASTER FUNCTIONS (VIEW CENSUS CONCEPTS)
'''
filter_words       = ['Presence of Own Children']
blocked_words      = ['(']

json_dictionary, group_dictionary, acs_dataframe = view_acs_codes(json_url, filter_words, blocked_words)

with option_context('display.max_colwidth', None): 
    print(acs_dataframe)

JSON dictionary returned
Group dictionary returned
Group dictionary filtered and passed to dataframe
  Group Codes                                                                                                                       Concepts
0      B23003  Presence of Own Children Under 18 Years by Age of Own Children Under 18 Years by Employment Status for Females 20 to 64 Years
1      B11013                                                                      Subfamily Type by Presence of Own Children Under 18 Years
2      B23009                    Presence of Own Children Under 18 Years by Family Type by Number of Workers in Family in the Past 12 Months
3      B25012                                                                                Tenure by Families and Presence of Own Children
4      B23010                Presence of Own Children Under 18 Years in Married-Couple Families by Work Experience of Householder and Spouse
5      B23007                                        

In [445]:
'''
RUN MASTER FUNCTIONS (CREATE OUTPUT DATA)
'''
shp_path                                         = define_geography(map_name, gdb, focus_points, focus_point_name, selection_distance, shp_folder)
json_dictionary, group_dictionary, acs_dataframe = view_acs_codes(json_url, filter_words, blocked_words)
variable_codes, code_label_dictionary            = return_acs_dictionary(input_group_code, json_dictionary, group_dictionary)
json_list                                        = retrieve_acs_json(variable_codes, shp_path, api_key)

SMU Minneapolis
 >>> 813
﻿
C:\DALE\Personal\Minnesota\MyProject\Shapefiles\SMU_Minneapolis_Block_Groups.shp
JSON dictionary returned
Group dictionary returned
Group dictionary filtered and passed to dataframe
Call #1 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 1.43
 
Call #2 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 2.71
 
Call #3 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 3.95
 
Call #4 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 5.2
 
Call #5 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 6.49
 
Call #6 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 7.7
 
Call #7 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 8.96
 
Call #8 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 10.28
 
Call #9 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 11.54
 
Call #10 / 813
>>> Sent request
Checking status:
>>> 

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
geodataframe                                     = create_geodataframe(shp_path, keep_columns)
merge_geodataframe, output_shp_path              = merge_dataframe_with_geometry(geodataframe, census_dataframe, output_shp)

In [ ]:
##########################################################################################
##########################################################################################
##########################################################################################
##########################################################################################

In [ ]:
'''
1. Trim unnecessary columns from geodataframe 
2. Format output columns so that they have clear names 
3. Create function to group columns if needed 
'''